In [1]:
!pip install PyShEx --upgrade -q

In [2]:
from pyshex import ShExEvaluator, PrefixLibrary


In [3]:
schema = """
PREFIX ex: <http://ex.example/#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX school: <http://school.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

school:enrolleeAge xsd:integer MinInclusive 13 MaxInclusive 20 

school:Enrollee {
  foaf:age @school:enrolleeAge ;
  ex:hasGuardian IRI {1,2}
}
"""

In [4]:
rdf = """
PREFIX ex: <http://ex.example/#>
PREFIX inst: <http://example.com/users/>
PREFIX school: <http://school.example/#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

inst:Alice foaf:age 13 ;                     
  ex:hasGuardian inst:Person2, inst:Person3 .

inst:Bob foaf:age 15 ;                       
  ex:hasGuardian inst:Person4 .  

inst:Claire foaf:age 12 ;                     
  ex:hasGuardian inst:Person5 .               

inst:Don foaf:age 14 . 

inst:Eric foaf:age 20 ;
  ex:hasGuardian inst:PersonA, inst:PersonB, inst:PersonC .
"""

In [5]:
p = PrefixLibrary(rdf)
for result in ShExEvaluator(rdf=rdf, 
                            schema=schema, 
                            focus=[p.INST.Alice, p.INST.Bob, p.INST.Claire, p.INST.Don, p.INST.Eric], 
                            start=p.SCHOOL.Enrollee).evaluate():
    print(f"{result.focus}: {'Passing' if result.result else 'Failing'}")
    if not result.result:
        print(result.reason)

http://example.com/users/Alice: Passing
http://example.com/users/Bob: Passing
http://example.com/users/Claire: Failing
  Numeric value volation - minimum inclusive: 13 actual: 12
http://example.com/users/Don: Failing
---> Testing http://example.com/users/Don against http://school.example/#Enrollee 
    No matching triples found for predicate http://ex.example/#hasGuardian
http://example.com/users/Eric: Failing
---> Testing http://example.com/users/Eric against http://school.example/#Enrollee 
    3 triples exceeds max {1,2}
